# Things to consider for continue

1. simulation with rankig over time


In [62]:
from mma_ranking.ranking.preprocess import load_train_data, read_fights
from mma_ranking.ranking.network_ranking import (
    fighter_perfomance,
    calculate_rank,
    generate_ranks_weightclass,
)

# path = r"\data\all_fights_new.csv"
path = r"..\..\data\all_fights_2024-04-05.csv"
start_date = "2014-01-01"

# setting pandas to show all columns and rows
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)


In [63]:
df = load_train_data(path=path)
[["Date", "winner", "losser", "BELT", "Weight", "method", "value", "draw"]]
matches_df = df[df["Date"] > start_date]

# dictiorary of fights
matches_sorted = matches_df.sort_values(by="Date", ascending=True)

In [64]:
matches_sorted.head()

,Date,winner,losser,BELT,BOUNES,Weight,TIME_FORMAT,Method,Method_detail,Judge1_score,Judge2_score,Judge3_score,ROUND,TIME,fighter1,fighter2,fighter1_result,fighter2_result,method,value,adj,adj_val,draw
2882,2014-01-04,Katsunori Kikuno,Quinn Mulhern,NaN,NaN,Lightweight,3,Decision - Unanimous,NaN,27 - 30.,27 - 30.,27 - 30.,3,5:00,Katsunori Kikuno,Quinn Mulhern,W,L,Decision,5,0.2,1.0,False
2740,2014-01-04,Mairbek Taisumov,Tae Hyun Bang,NaN,NaN,Lightweight,3,Decision - Unanimous,NaN,27 - 30.,27 - 30.,27 - 30.,3,5:00,Mairbek Taisumov,Tae Hyun Bang,W,L,Decision,5,0.2,1.0,False
2737,2014-01-04,Dustin Kimura,Jon Delos Reyes,NaN,NaN,Bantamweight,3,Submission,Armbar From Bottom Guard,NaN,NaN,NaN,1,2:13,Dustin Kimura,Jon Delos Reyes,W,L,sub,7,0.2,1.4,False
2738,2014-01-04,Russell Doane,Leandro Issa,NaN,sub,Bantamweight,3,Submission,Triangle Choke From Bottom Guard Technical Sub...,NaN,NaN,NaN,2,4:59,Leandro Issa,Russell Doane,L,W,sub,7,0.2,1.4,False
2888,2014-01-04,Tatsuya Kawajiri,Sean Soriano,NaN,NaN,Featherweight,3,Submission,Rear Naked Choke,NaN,NaN,NaN,2,0:50,Tatsuya Kawajiri,Sean Soriano,W,L,sub,7,0.2,1.4,False


In [65]:
print("matches_df.shape", matches_df.shape)
print(f"max date: {matches_df['Date'].dt.date.max()}")
print(f"min date: {matches_df['Date'].dt.date.min()}")

matches_df.shape (5078, 23)
max date: 2024-03-23
min date: 2014-01-04


# Network analysis


In [66]:
rankings_df = generate_ranks_weightclass(df)
rankings_df.head()

,fighter,page_rank,run_date,perf_rat,n_fights,adj_p_rank,weight,wight_class_rank
0,Marlon Vera,0.006149,2024-03-23,0.714286,7.0,0.004392,Bantamweight,1
1,Aljamain Sterling,0.005137,2024-03-23,0.750000,4.0,0.003853,Bantamweight,2
2,Sean O'Malley,0.003799,2024-03-23,1.000000,7.0,0.003799,Bantamweight,3
3,Merab Dvalishvili,0.003686,2024-03-23,1.000000,5.0,0.003686,Bantamweight,4
4,Deiveson Figueiredo,0.003336,2024-03-23,1.000000,1.0,0.003336,Bantamweight,5


In [67]:
rankings_df[rankings_df["fighter"] == "Frankie Edgar"]

,fighter,page_rank,run_date,perf_rat,n_fights,adj_p_rank,weight,wight_class_rank
122,Frankie Edgar,0.002716,2024-03-23,0.0,2.0,0.0,Bantamweight,82


In [68]:
weight_class_list = matches_df["Weight"].unique()
weight_class_list

array(['Bantamweight', 'Flyweight', 'Middleweight', "Women'sFlyweight",
       'Featherweight', 'Lightweight', 'Heavyweight',
       "Women's Bantamweight", "Women's Strawweight", 'Welterweight',
       'Light Heavyweight', 'other', "Women's Featherweight"],
      dtype=object)

In [86]:
run_date = "2024-03-23"
weight_class = weight_class_list[0]

In [87]:
# updating the rank based on multiplying page_rank and weight class page_rank

In [88]:
# Create a dictionary from the rankings for quick lookup and update
for weight_class_ in weight_class_list:
    print(f"weight_class: {weight_class_}")
    _df = rankings_df[
            (rankings_df["weight"] == weight_class_) & (rankings_df["run_date"] == run_date)
        ]
    # update and sort the rankings for each weight class
    _df.loc[:,'wight_class_rank'] = range(1, len(_df) + 1)    

weight_class: Bantamweight
weight_class: Flyweight
weight_class: Middleweight
weight_class: Women'sFlyweight
weight_class: Featherweight
weight_class: Lightweight
weight_class: Heavyweight
weight_class: Women's Bantamweight
weight_class: Women's Strawweight
weight_class: Welterweight
weight_class: Light Heavyweight
weight_class: other
weight_class: Women's Featherweight


In [89]:
# check if there are any duplicates
_df = rankings_df[
            (rankings_df["weight"] == weight_class) & (rankings_df["run_date"] == run_date)
        ]
_df[_df.duplicated(subset=["wight_class_rank"])].head()

,fighter,page_rank,run_date,perf_rat,n_fights,adj_p_rank,weight,wight_class_rank
65,Ramon Taveras,0.000068,2024-03-23,1.0,1.0,0.000068,Bantamweight,65
83,JP Buys,0.000035,2024-03-23,0.0,2.0,0.000000,Bantamweight,82
84,Cameron Else,0.000035,2024-03-23,0.0,1.0,0.000000,Bantamweight,82
85,Kris Moutinho,0.000035,2024-03-23,0.0,2.0,0.000000,Bantamweight,82
86,Serhiy Sidey,0.000035,2024-03-23,0.0,1.0,0.000000,Bantamweight,82


In [90]:
fighter_rankings = (
    rankings_df[
        (rankings_df["weight"] == weight_class) & (rankings_df["run_date"] == run_date)
    ][["fighter", "wight_class_rank", "adj_p_rank"]]
    .set_index("fighter")
    .to_dict("index")
)

fighter_rankings.get("Beneil Dariush", {"rank": 0})

{'rank': 0}

In [91]:
# get the fighter ranking from the dictionary
fighter_rankings.get("Beneil Dariush", {"rank": 0})

{'rank': 0}

In [92]:
my_dict = {'apple': 3, 'banana': 1, 'cherry': 2}
sorted_dict = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse=False))
sorted_dict

{'banana': 1, 'cherry': 2, 'apple': 3}

In [93]:
import pandas as pd

# "adj_p_rank"
epsilon = 0.000001

# "wight_class_rank"
for _, match in matches_sorted.iterrows():
    winner = match["winner"]
    loser = match["losser"]
    date = match["Date"]

    if match["draw"]:
        # Skip if it's a draw since nobody wins or loses
        continue
    if winner not in fighter_rankings or loser not in fighter_rankings:
        # Skip if either fighter is not in the rankings
        continue
    # Check if the winner's page_rank is less than the loser's
    if fighter_rankings[winner]["wight_class_rank"] >= fighter_rankings[loser]["wight_class_rank"]:

        winner_rank = fighter_rankings.get(winner, {"rank": 0})["wight_class_rank"]
        loser_rank = fighter_rankings.get(loser, {"rank": 0})["wight_class_rank"]
        print("date: {}".format(date))
        print(
            "Before change --> winner name {} --> rank: {} \n loser name {} --> rank: {}".format(
                winner, winner_rank, loser, loser_rank
            )
        )
        print("-------------------")

        # Update the winner's page_rank to match the loser's
        fighter_rankings[winner]["adj_p_rank"] = (
            fighter_rankings[loser]["adj_p_rank"] + epsilon
        )
        # Ensure winner's rank is better (lower) than the loser's
        # We find the next better rank that is not occupied by another fighter
        fighter_rankings[winner]["wight_class_rank"] = fighter_rankings[loser][
            "wight_class_rank"
        ]
        potential_new_rank = fighter_rankings[loser]["wight_class_rank"] + 1
                
        # update all the ranking higher than the loser
        for fighter in fighter_rankings.keys():
            figher_rank_ = fighter_rankings.get(fighter, {"rank": 0})["wight_class_rank"]
            
            if figher_rank_ >= potential_new_rank:
                print(
                    "fighter: {} rank: {}  --> {}".format(
                        fighter, figher_rank_, figher_rank_+1
                    )
                )
                fighter_rankings[fighter]["wight_class_rank"] += 1                    
        
        fighter_rankings[loser]["wight_class_rank"] = potential_new_rank
        winner_rank = fighter_rankings.get(winner, {"rank": 0})["wight_class_rank"]
        loser_rank = fighter_rankings.get(loser, {"rank": 0})["wight_class_rank"]
        print("-------------------")
        print(
            "After change --> winner name {} --> rank: {} \n loser name {} --> rank: {}".format(
                winner, winner_rank, loser, loser_rank
            )
        ) 
        fighter_rankings = dict(sorted(fighter_rankings.items(), key=lambda item: item[1]['wight_class_rank']))     

        
        print("\n-------------------\n")

    # Convert the updated rankings back to a DataFrame
    updated_rankings = pd.DataFrame.from_dict(
        fighter_rankings, orient="index"
    ).reset_index()
    updated_rankings.columns = ["Name", "rank", "page_rank"]

    # Sort by 'rank' to get the final rankings
    final_rankings = updated_rankings.sort_values(by="rank").reset_index(drop=True)

    # Adjust the rank numbers to be sequential after changes
    # final_rankings["rank"] = range(1, len(final_rankings) + 1)
    updated_rankings = updated_rankings.sort_values(by=["rank"]).sort_values(by=["page_rank"], ascending=False)
    # updated_rankings["rank"] = range(1, len(updated_rankings) + 1)

date: 2014-05-10 00:00:00
Before change --> winner name Johnny Eduardo --> rank: 82 
 loser name Eddie Wineland --> rank: 82
-------------------
-------------------
After change --> winner name Johnny Eduardo --> rank: 82 
 loser name Eddie Wineland --> rank: 83

-------------------

date: 2014-11-22 00:00:00
Before change --> winner name Frankie Edgar --> rank: 82 
 loser name Cub Swanson --> rank: 82
-------------------
fighter: Eddie Wineland rank: 83  --> 84
-------------------
After change --> winner name Frankie Edgar --> rank: 82 
 loser name Cub Swanson --> rank: 83

-------------------

date: 2015-11-07 00:00:00
Before change --> winner name Thomas Almeida --> rank: 82 
 loser name Anthony Birchak --> rank: 82
-------------------
fighter: Cub Swanson rank: 83  --> 84
fighter: Eddie Wineland rank: 84  --> 85
-------------------
After change --> winner name Thomas Almeida --> rank: 82 
 loser name Anthony Birchak --> rank: 83

-------------------

date: 2016-01-17 00:00:00
Befor

In [99]:
updated_rankings.sort_values(by=["rank"]).head(20)

,Name,rank,page_rank
0,Cory Sandhagen,1,0.004395
1,Sean O'Malley,2,0.004395
2,Marlon Vera,3,0.004394
3,Deiveson Figueiredo,4,0.004396
4,Rob Font,5,0.004395
5,Jonathan Martinez,6,0.004395
6,Adrian Yanez,7,0.004394
7,Aoriqileng,8,0.004395
8,Daniel Marcos,9,0.004394
9,Davey Grant,10,0.004393


In [82]:
def display_matches(fighter_name, matches_df):
    fighter_matches = matches_df[
        (matches_df["winner"] == fighter_name) | (matches_df["losser"] == fighter_name)
    ]
    return fighter_matches


In [85]:
display_matches("Benoit Saint Denis", matches_df)

,Date,winner,losser,BELT,BOUNES,Weight,TIME_FORMAT,Method,Method_detail,Judge1_score,Judge2_score,Judge3_score,ROUND,TIME,fighter1,fighter2,fighter1_result,fighter2_result,method,value,adj,adj_val,draw
7435,2024-03-09,Dustin Poirier,Benoit Saint Denis,NaN,fight,Lightweight,5,KO/TKO,Punch to Head At Distance,NaN,NaN,NaN,2,2:32,Dustin Poirier,Benoit Saint Denis,W,L,KO,7,1.0,7.0,False
179,2023-11-11,Benoit Saint Denis,Matt Frevola,NaN,perf,Lightweight,3,KO/TKO,Kick to Head At Distance,NaN,NaN,NaN,1,1:31,Matt Frevola,Benoit Saint Denis,L,W,KO,7,1.0,7.0,False
7489,2023-09-02,Benoit Saint Denis,Thiago Moises,NaN,fight,Lightweight,3,KO/TKO,Punches to Head On Ground,NaN,NaN,NaN,2,4:44,Benoit Saint Denis,Thiago Moises,W,L,KO,7,1.0,7.0,False
7215,2023-07-01,Benoit Saint Denis,Ismael Bonfim,NaN,NaN,Lightweight,3,Submission,Rear Naked Choke,NaN,NaN,NaN,1,4:48,Ismael Bonfim,Benoit Saint Denis,L,W,sub,7,1.0,7.0,False
7156,2022-09-03,Benoit Saint Denis,Gabriel Miranda,NaN,perf,Lightweight,3,KO/TKO,Punch to Head At Distance,NaN,NaN,NaN,2,0:16,Benoit Saint Denis,Gabriel Miranda,W,L,KO,7,1.0,7.0,False
6893,2022-06-04,Benoit Saint Denis,Niklas Stolze,NaN,NaN,Lightweight,3,Submission,Rear Naked Choke,NaN,NaN,NaN,2,1:32,Niklas Stolze,Benoit Saint Denis,L,W,sub,7,1.0,7.0,False
6382,2021-10-30,Elizeu Zaleski dos Santos,Benoit Saint Denis,NaN,NaN,Welterweight,3,Decision - Unanimous,NaN,26 - 29.,26 - 29.,26 - 29.,3,5:00,Elizeu Zaleski dos Santos,Benoit Saint Denis,W,L,Decision,5,1.0,5.0,False


In [17]:
LONG_FORMAT_DATA_START = "2020-01-01"
now = datetime.now()
formatted_date_time = now.strftime("%Y%m%d_%H%M")
print(formatted_date_time)
for dt in df["Date"].unique():
    if dt <= datetime.strptime(LONG_FORMAT_DATA_START, "%Y-%m-%d"):
        continue
    _df = df[df["Date"] <= dt]
    print(_df.shape, "     ", dt)
    df_w_rank_all = generate_ranks_weightclass(
        _df, file_name=f"rank_long_{formatted_date_time}.csv"
    )

20240325_1914
(20431, 22)       2024-03-16 00:00:00
(20418, 22)       2024-03-09 00:00:00
(20404, 22)       2024-03-02 00:00:00
(20393, 22)       2024-02-24 00:00:00
(20381, 22)       2024-02-17 00:00:00
(20369, 22)       2024-02-10 00:00:00
(20355, 22)       2024-02-03 00:00:00
(20342, 22)       2024-01-20 00:00:00
(20330, 22)       2024-01-13 00:00:00
(20319, 22)       2023-12-16 00:00:00
(20307, 22)       2023-12-09 00:00:00
(20296, 22)       2023-12-02 00:00:00
(20284, 22)       2023-11-18 00:00:00
(20270, 22)       2023-11-11 00:00:00
(20256, 22)       2023-11-04 00:00:00
(20245, 22)       2023-10-21 00:00:00
(20232, 22)       2023-10-14 00:00:00
(20221, 22)       2023-10-07 00:00:00
(20211, 22)       2023-09-23 00:00:00
(20200, 22)       2023-09-16 00:00:00
(20188, 22)       2023-09-09 00:00:00
(20176, 22)       2023-09-02 00:00:00
(20165, 22)       2023-08-26 00:00:00
(20152, 22)       2023-08-19 00:00:00
(20140, 22)       2023-08-12 00:00:00
(20127, 22)       2023-08-05 00:00:0

In [18]:
df_long = pd.read_csv(f"rank_long_{formatted_date_time}.csv")
df_long = df_long[df_long.wight_class_rank != "wight_class_rank"]
numeric_columns = [
    "page_rank",
    "perf_rat",
    "n_fights",
    "adj_p_rank",
    "wight_class_rank",
]
df_long[numeric_columns] = df_long[numeric_columns].apply(pd.to_numeric)
df_long["run_date"] = pd.to_datetime(df_long["run_date"])  # .dt.date

df_long["run_date"].unique()

<DatetimeArray>
['2024-03-16 00:00:00', '2024-03-09 00:00:00', '2024-03-02 00:00:00',
 '2024-02-24 00:00:00', '2024-02-17 00:00:00', '2024-02-10 00:00:00',
 '2024-02-03 00:00:00', '2024-01-20 00:00:00', '2024-01-13 00:00:00',
 '2023-12-16 00:00:00',
 ...
 '2020-05-13 00:00:00', '2020-05-09 00:00:00', '2020-03-14 00:00:00',
 '2020-03-07 00:00:00', '2020-02-29 00:00:00', '2020-02-22 00:00:00',
 '2020-02-15 00:00:00', '2020-02-08 00:00:00', '2020-01-25 00:00:00',
 '2020-01-18 00:00:00']
Length: 178, dtype: datetime64[ns]

In [19]:
df_long.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130455 entries, 0 to 130631
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   fighter           130455 non-null  object        
 1   page_rank         130455 non-null  float64       
 2   run_date          130455 non-null  datetime64[ns]
 3   perf_rat          130455 non-null  float64       
 4   n_fights          130455 non-null  float64       
 5   adj_p_rank        130455 non-null  float64       
 6   weight            130455 non-null  object        
 7   wight_class_rank  130455 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 9.0+ MB


In [20]:
df_long[df_long["fighter"].str.contains("Islam")]

,fighter,page_rank,run_date,perf_rat,n_fights,adj_p_rank,weight,wight_class_rank
355,Islam Makhachev,0.004331,2024-03-16,1.0,7.0,0.004331,Lightweight,1
1037,Islam Makhachev,0.004331,2024-03-16,1.0,2.0,0.004331,other,1
1102,Islam Makhachev,0.004337,2024-03-09,1.0,7.0,0.004337,Lightweight,1
2135,Islam Makhachev,0.004337,2024-03-09,1.0,2.0,0.004337,other,1
2309,Islam Makhachev,0.004366,2024-03-02,1.0,7.0,0.004366,Lightweight,1
...,...,...,...,...,...,...,...,...
128681,Islam Makhachev,0.001257,2020-02-22,1.0,7.0,0.001257,Lightweight,5
128955,Islam Makhachev,0.001262,2020-02-15,1.0,7.0,0.001262,Lightweight,5
129739,Islam Makhachev,0.001263,2020-02-08,1.0,7.0,0.001263,Lightweight,5
130158,Islam Makhachev,0.001288,2020-01-25,1.0,7.0,0.001288,Lightweight,5


In [26]:
_df = df_long[
    (df_long["weight"].str.contains("Lightweight")) & (df_long["wight_class_rank"] < 10)
].sort_values(["wight_class_rank", "run_date"])
_df.head()

,fighter,page_rank,run_date,perf_rat,n_fights,adj_p_rank,weight,wight_class_rank
130420,Michael Johnson,0.005937,2020-01-18,0.6,5.0,0.003562,Lightweight,1
130154,Michael Johnson,0.005929,2020-01-25,0.6,5.0,0.003558,Lightweight,1
129735,Michael Johnson,0.005803,2020-02-08,0.6,5.0,0.003482,Lightweight,1
128951,Michael Johnson,0.005767,2020-02-15,0.6,5.0,0.003460,Lightweight,1
128677,Michael Johnson,0.005705,2020-02-22,0.6,5.0,0.003423,Lightweight,1


In [24]:
# df_w_rank_all[(df_w_rank_all['n_fights']>1)].to_csv('w_ranking.csv')
# df_w_rank_all.to_csv("w_ranking.csv")

In [25]:
# ['Welterweight', 'Middleweight', 'Featherweight', 'Bantamweight',
#        "Women's Bantamweight", "Women'sFlyweight", 'Flyweight',
#        'Heavyweight', 'Lightweight', 'other', "Women's Strawweight",
#        'Light Heavyweight', "Women's Featherweight"]

In [ ]:
# df_w_rank_all[
#     (df_w_rank_all["n_fights"] > 3) & (df_w_rank_all["weight"] == "Featherweight")
# ].reset_index().head(25)

,index,fighter,page_rank,perf_rat,n_fights,adj_p_rank,weight
0,0,Max Holloway,0.004372,0.800000,5.0,0.003497,Featherweight
1,1,Ilia Topuria,0.003023,1.000000,5.0,0.003023,Featherweight
2,2,Alexander Volkanovski,0.003328,0.857143,7.0,0.002853,Featherweight
3,5,Movsar Evloev,0.001297,1.000000,5.0,0.001297,Featherweight
4,6,Edson Barboza,0.002323,0.500000,8.0,0.001161,Featherweight
5,7,Josh Emmett,0.001630,0.666667,6.0,0.001086,Featherweight
6,8,Cub Swanson,0.001973,0.500000,6.0,0.000986,Featherweight
7,9,Lerone Murphy,0.000908,1.000000,4.0,0.000908,Featherweight
8,10,Darren Elkins,0.001293,0.625000,8.0,0.000808,Featherweight
9,11,Calvin Kattar,0.001470,0.500000,4.0,0.000735,Featherweight


In [27]:
def pfp_performance(m):
    df_1 = (
        df[(df["Date"] >= dt - timedelta(weeks=52 * m))]
        .groupby(["fighter1", "fighter1_result"])["Weight"]
        .count()
        .unstack(level=-1)
        .fillna(0)
    )
    df_1.index.set_names(["fighter"], inplace=True)
    df_2 = (
        df[(df["Date"] >= dt - timedelta(weeks=52 * m))]
        .groupby(["fighter2", "fighter2_result"])["Weight"]
        .count()
        .unstack(level=-1)
        .fillna(0)
    )
    df_2.index.set_names(["fighter"], inplace=True)
    df_perf = df_1.add(df_2, fill_value=0)
    df_perf["n_fights"] = df_perf.sum(axis=1)
    df_perf["perf_rat"] = (df_perf["n_fights"] - df_perf["L"]) / df_perf["n_fights"]
    df_perf.columns = df_perf.columns.values
    df_perf = df_perf.sort_values(
        by=["perf_rat", "n_fights"], ascending=False
    ).reset_index(
        level=-1
    )  # [df_perf.n_fights>2]
    return df_perf

In [28]:
df_perf_pfp = pfp_performance(3)
df_pfp = pd.merge(
    df_ranked.reset_index().rename({"index": "fighter"}, axis=1),
    df_perf_pfp,
    on="fighter",
    how="inner",
)
df_pfp["adj_p_rank"] = df_pfp["page_rank"] * df_pfp["perf_rat"]
df_pfp.sort_values(by="adj_p_rank", ascending=False, inplace=True, ignore_index=True)
# df_pfp[(df_pfp['n_fights']>1)].to_csv('pfp_ranking.csv')
df_pfp.to_csv("pfp_ranking.csv")
df_pfp[(df_pfp["n_fights"] > 7)].head(25)

,fighter,page_rank,run_date,D,L,NC,W,n_fights,perf_rat,adj_p_rank
0,Amanda Nunes,0.009480,2024-03-16,0.0,5.0,0.0,44.0,49.0,0.897959,0.008513
1,Jon Jones,0.008069,2024-03-16,0.0,3.0,3.0,61.0,67.0,0.955224,0.007708
2,Israel Adesanya,0.008657,2024-03-16,0.0,5.0,0.0,33.0,38.0,0.868421,0.007518
3,Francis Ngannou,0.007647,2024-03-16,0.0,6.0,0.0,35.0,41.0,0.853659,0.006528
4,Stipe Miocic,0.008048,2024-03-16,0.0,12.0,0.0,42.0,54.0,0.777778,0.006259
5,Matt Hamill,0.008333,2024-03-16,0.0,15.0,0.0,30.0,45.0,0.666667,0.005555
6,Derrick Lewis,0.007673,2024-03-16,0.0,22.0,0.0,51.0,73.0,0.698630,0.005360
7,Curtis Blaydes,0.006719,2024-03-16,0.0,10.0,3.0,33.0,46.0,0.782609,0.005259
8,Alex Pereira,0.005267,2024-03-16,0.0,1.0,0.0,8.0,9.0,0.888889,0.004682
9,Kamaru Usman,0.004984,2024-03-16,0.0,3.0,0.0,44.0,47.0,0.936170,0.004666


# Second method


In [49]:
# Function to create a directed graph from the matches DataFrame using nx.from_pandas_edgelist
def create_graph_from_matches(df, weight="value"):
    # Create a graph from the DataFrame using 'nx.from_pandas_edgelist'
    G = nx.from_pandas_edgelist(
        df, source="loser", target="winner", edge_attr=weight, create_using=nx.DiGraph()
    )
    return G


# Function to rank fighters for each weight class and date using nx.from_pandas_edgelist
def rank_fighters(df, weight_classes):
    rankings = pd.DataFrame()

    for weight_class in weight_classes:
        for current_date in pd.to_datetime(df["Date"]).dt.date.unique():
            # Ensure date filtering is correct after conversion
            current_date = current_date.strftime("%Y-%m-%d")

            filtered_matches = filter_matches_by_date_and_class(
                df, current_date, weight_class
            )

            # If the DataFrame is empty for this date and class, skip to the next iteration
            if filtered_matches.empty:
                continue

            # Here, we adjust the DataFrame to have 'loser' and 'winner' columns as required by nx.from_pandas_edgelist
            if "draw" in filtered_matches.columns:
                draws = filtered_matches[filtered_matches["draw"]]
                wins = filtered_matches[~filtered_matches["draw"]]
                # Duplicate the rows for draws with the 'loser' and 'winner' swapped
                draws_swapped = draws.rename(
                    columns={"winner": "loser", "loser": "winner"}
                )
                filtered_matches = pd.concat(
                    [wins, draws, draws_swapped], ignore_index=True
                )

            # Create a directed graph from the matches DataFrame using the utility function
            G = create_graph_from_matches(filtered_matches)

            # Calculate PageRank, considering edge weights
            pagerank = nx.pagerank(G, weight="value")
            # Prepare rankings data for the current date and weight class
            current_rankings = (
                pd.DataFrame(
                    {
                        "date": current_date,
                        "weight_class": weight_class,
                        "fighter": list(pagerank.keys()),
                        "score": list(pagerank.values()),
                    }
                )
                .sort_values(by="score", ascending=False)
                .reset_index(drop=True)
            )
            # Append current date rankings for the weight class
            rankings = pd.concat([rankings, current_rankings], axis=0)

    return rankings


# The rest of your code for adjusting the DataFrame, defining weight classes, and ranking fighters would remain the same.
# Define the weight classes from the data
weight_classes = matches_df["Weight"].unique()

# Adjust matches DataFrame to account for weight classes and wins across classes
matches_df_adjusted = adjust_for_weight_classes(matches_df)

# Rank fighters and display rankings for each weight class and date
ranked_fighters_over_time = rank_fighters(matches_df_adjusted, weight_classes)

# Example: Display rankings for the latest date in the dataset and a specific weight class
latest_date = ranked_fighters_over_time["date"].max()
specific_weight_class = "Lightweight"
ranked_fighters_over_time[
    (ranked_fighters_over_time["date"] == latest_date)
    & (ranked_fighters_over_time["weight_class"] == specific_weight_class)
].head(20)

,date,weight_class,fighter,score
0,2024-03-30,Lightweight,Nate Diaz,0.002934
1,2024-03-30,Lightweight,Glaico Franca Moreira,0.002934
2,2024-03-30,Lightweight,Martin Bravo,0.002934
3,2024-03-30,Lightweight,Michael Chiesa,0.002934
4,2024-03-30,Lightweight,Conor McGregor,0.002934
5,2024-03-30,Lightweight,Guy Mezger,0.002934
6,2024-03-30,Lightweight,Efrain Escudero,0.002929
7,2024-03-30,Lightweight,Jonathan Brookins,0.002929
8,2024-03-30,Lightweight,Ryan Hall,0.002929
9,2024-03-30,Lightweight,Michael Trizano,0.002929
